In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import tarfile
import io
import time
from collections import Counter

class NetworkAnalyzer:
    def __init__(self):
        """Khởi tạo class NetworkAnalyzer"""
        self.data = None
        self.graph = None
        self.metrics = {}

    def download_and_read_data(self, url=None):
        """
        Tải và đọc dữ liệu từ KONECT
        Args:
            url: URL của dataset (tùy chọn)
        Returns:
            DataFrame: Chứa thông tin về các cạnh
        """
        if url is None:
            url = "http://konect.cc/files/download.tsv.dblp_coauthor.tar.bz2"

        print("Đang tải và đọc dữ liệu...")

        try:
            response = requests.get(url)
            if response.status_code == 200:
                tar = tarfile.open(fileobj=io.BytesIO(response.content), mode='r:bz2')
                data_file = [f for f in tar.getmembers() if f.name.endswith('out.dblp_coauthor')][0]
                data_content = tar.extractfile(data_file).read().decode('utf-8')

                # Đọc dữ liệu từng dòng
                rows = []
                skipped = 0
                for line in data_content.split('\n'):
                    if line and not line.startswith('%'):
                        try:
                            # Tách dòng thành các phần
                            parts = line.strip().split()
                            if len(parts) >= 4:  # Kiểm tra có đủ 4 cột
                                author1 = int(parts[0])
                                author2 = int(parts[1])
                                weight = float(parts[2])
                                timestamp = int(parts[3])

                                rows.append({
                                    'author1': author1,
                                    'author2': author2,
                                    'weight': weight,
                                    'timestamp': timestamp
                                })
                            else:
                                skipped += 1
                                print(f"Dòng thiếu dữ liệu: {line}")

                        except (ValueError, IndexError) as e:
                            skipped += 1
                            print(f"Lỗi xử lý dòng: {line} - {str(e)}")
                            continue

                # Tạo DataFrame
                df = pd.DataFrame(rows)

                # In thông tin kiểm tra
                print(f"\nKết quả đọc dữ liệu:")
                print(f"- Số cạnh đã đọc thành công: {len(df):,}")
                print(f"- Số dòng bị bỏ qua: {skipped:,}")
                print("\nMẫu dữ liệu:")
                print(df.head())
                print("\nThông tin các cột:")
                print(df.dtypes)
                print("\nThống kê:")
                print(df.describe())

                tar.close()
                self.data = df
                return True

            else:
                raise Exception(f"Lỗi khi tải dữ liệu: Status code {response.status_code}")
        except Exception as e:
            print(f"Lỗi: {str(e)}")
            return False

    def create_graph(self):
        """Tạo đồ thị từ DataFrame"""
        if self.data is None:
            print("Chưa có dữ liệu để tạo đồ thị")
            return False

        print("Đang tạo đồ thị...")
        try:
            # Chỉ sử dụng các cột cần thiết cho đồ thị
            self.graph = nx.from_pandas_edgelist(
                self.data,
                'author1',
                'author2',
                edge_attr=['weight', 'timestamp'],  # Thêm timestamp vào thuộc tính cạnh
                create_using=nx.Graph()
            )

            print(f"Đã tạo đồ thị với {self.graph.number_of_nodes():,} nodes và "
                  f"{self.graph.number_of_edges():,} edges")
            return True

        except Exception as e:
            print(f"Lỗi khi tạo đồ thị: {str(e)}")
            return False

    def analyze_network(self):
        """Phân tích các đặc trưng của mạng"""
        if self.graph is None:
            print("Chưa có đồ thị để phân tích")
            return False

        print("\nĐang phân tích mạng...")

        # 1. Thông tin cơ bản
        start_time = time.time()
        self.metrics['nodes'] = self.graph.number_of_nodes()
        self.metrics['edges'] = self.graph.number_of_edges()
        self.metrics['density'] = nx.density(self.graph)
        print(f"Thời gian tính thông tin cơ bản: {time.time() - start_time:.2f}s")

        # 2. Degree metrics
        start_time = time.time()
        degrees = dict(self.graph.degree())
        weighted_degrees = {
            node: sum(dict(self.graph[node]).values())
            for node in self.graph.nodes()
        }

        self.metrics['degrees'] = degrees
        self.metrics['weighted_degrees'] = weighted_degrees
        self.metrics['avg_degree'] = np.mean(list(degrees.values()))
        self.metrics['max_degree'] = max(degrees.values())
        self.metrics['avg_weighted_degree'] = np.mean(list(weighted_degrees.values()))
        self.metrics['max_weighted_degree'] = max(weighted_degrees.values())
        print(f"Thời gian tính degree metrics: {time.time() - start_time:.2f}s")

        # 3. Centrality measures
        # Degree Centrality
        start_time = time.time()
        dc = nx.degree_centrality(self.graph)
        self.metrics['degree_centrality'] = {
            'values': dc,
            'max': max(dc.values()),
            'avg': np.mean(list(dc.values())),
            'top': sorted(dc.items(), key=lambda x: x[1], reverse=True)[:10]
        }
        print(f"Thời gian tính Degree Centrality: {time.time() - start_time:.2f}s")

        # Betweenness Centrality
        start_time = time.time()
        bc = nx.betweenness_centrality(self.graph)
        self.metrics['betweenness_centrality'] = {
            'values': bc,
            'max': max(bc.values()),
            'avg': np.mean(list(bc.values())),
            'top': sorted(bc.items(), key=lambda x: x[1], reverse=True)[:10]
        }
        print(f"Thời gian tính Betweenness Centrality: {time.time() - start_time:.2f}s")

        # Closeness Centrality
        start_time = time.time()
        cc = nx.closeness_centrality(self.graph)
        self.metrics['closeness_centrality'] = {
            'values': cc,
            'max': max(cc.values()),
            'avg': np.mean(list(cc.values())),
            'top': sorted(cc.items(), key=lambda x: x[1], reverse=True)[:10]
        }
        print(f"Thời gian tính Closeness Centrality: {time.time() - start_time:.2f}s")

        # PageRank
        start_time = time.time()
        pr = nx.pagerank(self.graph)
        self.metrics['pagerank'] = {
            'values': pr,
            'max': max(pr.values()),
            'avg': np.mean(list(pr.values())),
            'top': sorted(pr.items(), key=lambda x: x[1], reverse=True)[:10]
        }
        print(f"Thời gian tính PageRank: {time.time() - start_time:.2f}s")

        return True

    def print_results(self):
        """In kết quả phân tích"""
        if not self.metrics:
            print("Chưa có kết quả phân tích")
            return

        print("\nKẾT QUẢ PHÂN TÍCH MẠNG ĐỒNG TÁC GIẢ")
        print("=" * 50)

        # 1. Thông tin cơ bản
        print("\n1. THÔNG TIN CƠ BẢN")
        print("-" * 30)
        print(f"Số lượng tác giả: {self.metrics['nodes']:,}")
        print(f"Số lượng cộng tác: {self.metrics['edges']:,}")
        print(f"Mật độ mạng: {self.metrics['density']:.6f}")
        print(f"Degree trung bình: {self.metrics['avg_degree']:.2f}")
        print(f"Degree lớn nhất: {self.metrics['max_degree']}")
        print(f"Weighted degree trung bình: {self.metrics['avg_weighted_degree']:.2f}")
        print(f"Weighted degree lớn nhất: {self.metrics['max_weighted_degree']}")

        # 2. Centrality Measures
        centrality_measures = {
            'Degree Centrality': 'degree_centrality',
            'Betweenness Centrality': 'betweenness_centrality',
            'Closeness Centrality': 'closeness_centrality',
            'PageRank': 'pagerank'
        }

        print("\n2. CENTRALITY MEASURES")
        print("-" * 30)
        for name, measure in centrality_measures.items():
            print(f"\n{name}:")
            print(f"- Giá trị trung bình: {self.metrics[measure]['avg']:.6f}")
            print(f"- Giá trị lớn nhất: {self.metrics[measure]['max']:.6f}")
            print("\nTop 5 tác giả:")
            for author, score in self.metrics[measure]['top'][:5]:
                print(f"  {author}: {score:.6f}")


def main():
    # Khởi tạo analyzer
    analyzer = NetworkAnalyzer()

    # Tải và đọc dữ liệu
    if not analyzer.download_and_read_data():
        return

    # Tạo đồ thị
    if not analyzer.create_graph():
        return

     # Phân tích mạng
    if not analyzer.analyze_network():
        return

    # In và trực quan hóa kết quả
    analyzer.print_results()

if __name__ == "__main__":
    main()

# Yêu cầu
## 1. Xây dựng phương thức: visualize_metrics trực quan hóa các số đo bằng biểu đồ
Tạo figure với 2x2 subplots:
- Degree Distribution
- Centrality Comparison
- Top nodes comparison
....

## 2. Từ các số đo hãy đưa ra nhận định cơ bản về mạng xã hội
## 3. Thay đổi cách download dữ liệu trực tiếp thành việc đọc dữ liệu từ file:
o	File được lưu trữ tại Google Drive (dành cho Google Colab)
o	File được lưu trữ trực tiếp trên máy